In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('aggregate_data.csv')
to_drop = ['Company(t-30)', 'Company(t)', 'EffectiveSentimentPolarity(t)']
           
data = data.drop(to_drop, axis=1)
data.head()

/var/folders/_4/kpr2wwvs1p34_pmnnchyg7lm0000gn/T/ipykernel_59750/3272126367.py:1: DtypeWarning: Columns (0,61) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('aggregate_data.csv')


,Close(t-30),EffectiveSentimentPolarity(t-30),Close(t-29),EffectiveSentimentPolarity(t-29),Close(t-28),EffectiveSentimentPolarity(t-28),Close(t-27),EffectiveSentimentPolarity(t-27),Close(t-26),EffectiveSentimentPolarity(t-26),...,EffectiveSentimentPolarity(t-5),Close(t-4),EffectiveSentimentPolarity(t-4),Close(t-3),EffectiveSentimentPolarity(t-3),Close(t-2),EffectiveSentimentPolarity(t-2),Close(t-1),EffectiveSentimentPolarity(t-1),Close(t)
0,96.424286,0.0,95.747147,0.0,96.610001,0.0,97.205711,0.0,94.677139,0.0,...,0.0,91.558571,0.0,89.728569,0.0,89.958572,0.0,90.680000,0.0,92.827141
1,95.747147,0.0,96.610001,0.0,97.205711,0.0,94.677139,0.0,94.370003,0.0,...,0.0,89.728569,0.0,89.958572,0.0,90.680000,0.0,92.827141,0.0,92.087143
2,96.610001,0.0,97.205711,0.0,94.677139,0.0,94.370003,0.0,95.684288,0.0,...,0.0,89.958572,0.0,90.680000,0.0,92.827141,0.0,92.087143,0.0,90.377144
3,97.205711,0.0,94.677139,0.0,94.370003,0.0,95.684288,0.0,97.568573,0.0,...,0.0,90.680000,0.0,92.827141,0.0,92.087143,0.0,90.377144,0.0,87.120003
4,94.677139,0.0,94.370003,0.0,95.684288,0.0,97.568573,0.0,98.754288,0.0,...,0.0,92.827141,0.0,92.087143,0.0,90.377144,0.0,87.120003,0.0,90.575714


### With ESP

In [3]:
# divide into train and test
div = int( len(data)*0.3 )
data_train = data.iloc[:div, :]
data_test = data.iloc[div:, :]

# divide into X and y
X_train = data_train.iloc[:, :-1].values
y_train = data_train.iloc[:, -1].values
X_test = data_test.iloc[:, :-1].values
y_test = data_test.iloc[:, -1].values

In [4]:
#create scalers 
from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
X_train_scaled = x_scaler.fit_transform(X_train)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1))

#### Linear Regression

In [5]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train_scaled)

LinearRegression()

In [6]:
lr_model.score(X_train_scaled,y_train_scaled)

0.999926925544369

In [7]:
X_test_scaled = x_scaler.transform(X_test)
y_hat = lr_model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat)

In [8]:
from sklearn.metrics import mean_squared_error
import math

math.sqrt(mean_squared_error(y_test, y_hat_descaled))

3.718030648507633

#### Ridge Regression

In [10]:
from sklearn.linear_model import Ridge

rg_model = Ridge()
rg_model.fit(X_train_scaled, y_train_scaled)

print( rg_model.score(X_train_scaled,y_train_scaled) )

X_test_scaled = x_scaler.transform(X_test)
y_hat = rg_model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat)

from sklearn.metrics import mean_squared_error
import math

math.sqrt(mean_squared_error(y_test, y_hat_descaled))

0.9998100621850993


5.707197608548323

#### Lasso Regression

In [11]:
from sklearn.linear_model import Lasso

ls_model = Lasso(alpha=0.00001)
ls_model.fit(X_train_scaled, y_train_scaled)

print( ls_model.score(X_train_scaled,y_train_scaled) )

X_test_scaled = x_scaler.transform(X_test)
y_hat = ls_model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat.reshape(-1,1))

from sklearn.metrics import mean_squared_error
import math

math.sqrt(mean_squared_error(y_test, y_hat_descaled))

0.999570466226777


/Users/jarvis/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.424e-01, tolerance: 6.729e-02
  model = cd_fast.enet_coordinate_descent(


7.815717166132901

#### SGD Regressor

In [12]:
from sklearn.linear_model import SGDRegressor

sgd_model = SGDRegressor(alpha=0.00001)
sgd_model.fit(X_train_scaled, y_train_scaled)

print( sgd_model.score(X_train_scaled,y_train_scaled) )

X_test_scaled = x_scaler.transform(X_test)
y_hat = sgd_model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat.reshape(-1,1))

from sklearn.metrics import mean_squared_error
import math

math.sqrt(mean_squared_error(y_test, y_hat_descaled))

0.9993381413401909


/Users/jarvis/miniforge3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


33.35913095081706

#### Decision tree regressor

In [13]:
from sklearn.tree import DecisionTreeRegressor

dtr_model = DecisionTreeRegressor(max_depth=8, random_state=0)
dtr_model.fit(X_train_scaled, y_train_scaled)

DecisionTreeRegressor(max_depth=8, random_state=0)

In [14]:
dtr_model.score(X_train_scaled,y_train_scaled)

0.9999587866743497

In [15]:
X_test_scaled = x_scaler.transform(X_test)
y_hat = dtr_model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat.reshape(-1,1))

In [16]:
math.sqrt(mean_squared_error(y_test, y_hat_descaled))

44.103015522275356

#### Support Vector Regressor

In [17]:
from sklearn.svm import SVR

svr_model = SVR(max_iter=10000)
svr_model.fit(X_train_scaled, y_train_scaled)

/Users/jarvis/miniforge3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(max_iter=10000)

In [18]:
svr_model.score(X_train_scaled,y_train_scaled)

0.9594338047508056

In [19]:
X_test_scaled = x_scaler.transform(X_test)
y_hat = svr_model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat.reshape(-1,1))

In [20]:
math.sqrt(mean_squared_error(y_test, y_hat_descaled))

7893.736895246721

#### LSTM

In [21]:
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = x_scaler.transform(X_test)
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [22]:
X_train_scaled.shape

(33900, 1, 60)

In [23]:
y_test_scaled = y_scaler.transform(y_test.reshape(-1,1))

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# design network
model = Sequential()
model.add(LSTM(100, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(X_train_scaled, y_train_scaled, epochs=15, batch_size=72, validation_data=(X_test_scaled, y_test_scaled), verbose=1, shuffle=False)

Metal device set to: Apple M1
Epoch 1/15


2022-04-25 08:07:23.641656: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-25 08:07:23.641740: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-25 08:07:23.777319: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-25 08:07:24.189135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-25 08:07:24.299720: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-25 08:07:24.358668: I tensorflow/core/grappler/o

469/471 [============================>.] - ETA: 0s - loss: 0.0093

2022-04-25 08:07:27.311283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-25 08:07:27.352128: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


471/471 [==============================] - 7s 13ms/step - loss: 0.0093 - val_loss: 0.0019
Epoch 2/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0073 - val_loss: 6.0472e-04
Epoch 3/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0035 - val_loss: 8.8342e-04
Epoch 4/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0033 - val_loss: 0.0023
Epoch 5/15
471/471 [==============================] - 6s 13ms/step - loss: 0.0051 - val_loss: 0.0011
Epoch 6/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0073 - val_loss: 6.4082e-04
Epoch 7/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0059 - val_loss: 0.0016
Epoch 8/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0043 - val_loss: 0.0019
Epoch 9/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0021 - val_loss: 1.7277e-04
Epoch 10/15
471/471 [==============================] - 6s 12ms/step - loss: 0.0059 - v

In [25]:
yhat = model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat.reshape(-1,1))

2022-04-25 08:08:48.988664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-25 08:08:49.023709: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [26]:
math.sqrt(mean_squared_error(y_test, y_hat_descaled))

7893.736895246721

### Without ESP

In [27]:
to_drop = ['EffectiveSentimentPolarity(t-%d)'%i for i in range(1,31)]
data_wo = data.drop(to_drop, axis=1)
data_wo.head()

,Close(t-30),Close(t-29),Close(t-28),Close(t-27),Close(t-26),Close(t-25),Close(t-24),Close(t-23),Close(t-22),Close(t-21),...,Close(t-9),Close(t-8),Close(t-7),Close(t-6),Close(t-5),Close(t-4),Close(t-3),Close(t-2),Close(t-1),Close(t)
0,96.424286,95.747147,96.610001,97.205711,94.677139,94.370003,95.684288,97.568573,98.754288,99.968575,...,95.921425,95.257141,93.227142,91.167145,90.835716,91.558571,89.728569,89.958572,90.680000,92.827141
1,95.747147,96.610001,97.205711,94.677139,94.370003,95.684288,97.568573,98.754288,99.968575,100.272858,...,95.257141,93.227142,91.167145,90.835716,91.558571,89.728569,89.958572,90.680000,92.827141,92.087143
2,96.610001,97.205711,94.677139,94.370003,95.684288,97.568573,98.754288,99.968575,100.272858,100.300003,...,93.227142,91.167145,90.835716,91.558571,89.728569,89.958572,90.680000,92.827141,92.087143,90.377144
3,97.205711,94.677139,94.370003,95.684288,97.568573,98.754288,99.968575,100.272858,100.300003,99.814285,...,91.167145,90.835716,91.558571,89.728569,89.958572,90.680000,92.827141,92.087143,90.377144,87.120003
4,94.677139,94.370003,95.684288,97.568573,98.754288,99.968575,100.272858,100.300003,99.814285,100.012856,...,90.835716,91.558571,89.728569,89.958572,90.680000,92.827141,92.087143,90.377144,87.120003,90.575714


In [28]:
# divide into train and test
div = int( len(data)*0.3 )
data_train = data_wo.iloc[:div, :]
data_test = data_wo.iloc[div:, :]

# divide into X and y
X_train = data_train.iloc[:, :-1].values
y_train = data_train.iloc[:, -1].values
X_test = data_test.iloc[:, :-1].values
y_test = data_test.iloc[:, -1].values

In [29]:
x_scaler = MinMaxScaler()
X_train_scaled = x_scaler.fit_transform(X_train)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1))

In [30]:
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train_scaled)

LinearRegression()

In [31]:
X_test_scaled = x_scaler.transform(X_test)
y_hat = lr_model.predict(X_test_scaled)
y_hat_descaled = y_scaler.inverse_transform(y_hat)

In [32]:
math.sqrt(mean_squared_error(y_test, y_hat_descaled))

2.9642501944129185